In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler                   
from tensorflow.keras import Sequential                             
from tensorflow.keras.layers import LSTM,Dense,Dropout,Activation
import numpy as np
import plotly.graph_objects as go

In [14]:
df = pd.read_csv("Dataset_10Jun22.csv")
print(len(df))
df.head() 

1100


,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,SBIN,EQ,01-Jan-18,309.90,310.6,312.75,306.30,306.75,307.10,309.08,12182400,3.765315e+09,75769,5697288,46.77
1,SBIN,EQ,02-Jan-18,307.10,309.0,309.00,301.35,303.00,303.25,304.00,14797364,4.498328e+09,100195,5698196,38.51
2,SBIN,EQ,03-Jan-18,303.25,305.3,306.45,302.15,302.80,302.85,304.32,13857373,4.217070e+09,117207,6941140,50.09
3,SBIN,EQ,04-Jan-18,302.85,304.5,310.20,303.15,309.25,308.50,307.63,24517099,7.542211e+09,140513,10508054,42.86
4,SBIN,EQ,05-Jan-18,308.50,311.4,311.40,305.60,307.25,306.35,308.05,14580575,4.491544e+09,91829,4896408,33.58


In [15]:
Open_df = df[['Open Price','Total Traded Quantity','No. of Trades','Deliverable Qty']]

In [16]:
y_data = Open_df['Open Price']
X_data = Open_df[['Open Price','Total Traded Quantity','No. of Trades','Deliverable Qty']]

In [17]:
X_data.head()

,Open Price,Total Traded Quantity,No. of Trades,Deliverable Qty
0,310.6,12182400,75769,5697288
1,309.0,14797364,100195,5698196
2,305.3,13857373,117207,6941140
3,304.5,24517099,140513,10508054
4,311.4,14580575,91829,4896408


In [18]:
X_data_arr = np.asarray(X_data)
y_data_arr = np.asarray(y_data)

In [19]:
X_data_arr

array([[3.1060000e+02, 1.2182400e+07, 7.5769000e+04, 5.6972880e+06],
       [3.0900000e+02, 1.4797364e+07, 1.0019500e+05, 5.6981960e+06],
       [3.0530000e+02, 1.3857373e+07, 1.1720700e+05, 6.9411400e+06],
       ...,
       [4.6660000e+02, 1.7168871e+07, 1.9200200e+05, 6.5979930e+06],
       [4.6895000e+02, 1.2635601e+07, 1.5825900e+05, 5.4134140e+06],
       [4.6200000e+02, 8.6211040e+06, 1.2782800e+05, 3.0622010e+06]])

In [20]:
y_data_arr

array([310.6 , 309.  , 305.3 , ..., 466.6 , 468.95, 462.  ])

In [21]:
# Reshaping to automatically make it a 2d-array which is the input for sequential model
y_data_arr = y_data_arr.reshape(-1, 1)
y_data_arr

array([[310.6 ],
       [309.  ],
       [305.3 ],
       ...,
       [466.6 ],
       [468.95],
       [462.  ]])

In [22]:
# We scale the data to make the data modelling easier
x_scaler = MinMaxScaler(feature_range = (0, 1))
y_scaler = MinMaxScaler(feature_range = (0, 1))
x_scaled_data = x_scaler.fit_transform(X_data_arr)
y_scaled_data = y_scaler.fit_transform(y_data_arr)

In [23]:
# We find the index for train_test_split as spefied ratio by the user above in the parameters
train_test_split = 0.80
training_data_len = int(len(df) * train_test_split)
training_data_len

880

In [24]:
x_train_data = x_scaled_data[0:training_data_len,:]
print(" Shape of x_train_data")
print(x_train_data.shape)
y_train_data = y_scaled_data[0:training_data_len,:]
print(" Shape of y_train_data")
print(y_train_data.shape)

 Shape of x_train_data
(880, 4)
 Shape of y_train_data
(880, 1)


In [25]:
time_step = 30 #to predict the first test value, we need previous 30 days data
x_test_data = x_scaled_data[training_data_len-time_step:]
y_test_data = y_scaled_data[training_data_len-time_step:]
print(" Shape of x_test_data")
print(x_test_data.shape)
print(" Shape of y_test_data")
print(y_test_data.shape)

 Shape of x_test_data
(250, 4)
 Shape of y_test_data
(250, 1)


In [26]:
#To get the new datasets based on the sliding window
def create_multivariate_dataset(x_dataset, y_dataset, time_step = 30):
  dataX, dataY = [], []
  for i in range(len(x_dataset) - time_step) :
    dataX.append(x_dataset[i: (i + time_step), 0])
  for i in range(len(y_dataset) - time_step) :
    dataY. append (y_dataset[i + time_step, 0])
  return np.array(dataX), np.array(dataY)

In [27]:
# Splting dataset of time steps into train and test
X_train, y_train = create_multivariate_dataset(x_train_data, y_train_data, time_step)
X_test, y_test = create_multivariate_dataset(x_test_data, y_test_data, time_step)
print("Shape of Training dataset", X_train.shape, y_train.shape)
print("Shape of Testing dataset", X_test.shape, y_test.shape)

Shape of Training dataset (850, 30) (850,)
Shape of Testing dataset (220, 30) (220,)


In [28]:
# Converting 2D data into 3D data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#y_train & y_test are old values itself
print("Shape of Reshaped Training dataset", X_train.shape, y_train.shape)
print("Shape of Reshaped Testing dataset", X_test.shape, y_test.shape)

Shape of Reshaped Training dataset (850, 30, 1) (850,)
Shape of Reshaped Testing dataset (220, 30, 1) (220,)


In [29]:
dates = df['Date']

In [30]:
X_train.shape[1]

30

In [31]:
#Train the model

def build_model(batch_size, epochs):
  model = Sequential()
  model.add(LSTM(units=32, return_sequences=True, input_shape=(X_train.shape[1], 1)))
  model.add(LSTM(units=30, return_sequences=True))
  model.add(LSTM(units=15, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(1))
  model.compile(loss = 'mse',optimizer='adam')
  model.summary()

  print("Model is fitting...")
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=True)
  print("Model fitted!")
  return model


In [32]:
model = build_model(8, 50)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 30, 30)            7560      
                                                                 
 lstm_2 (LSTM)               (None, 15)                2760      
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense (Dense)               (None, 1)                 16        
                                                                 
Total params: 14,688
Trainable params: 14,688
Non-trainable params: 0
_________________________________________________________________
Model is fitting...
Epoch 1/50
107/107 [============

In [33]:
predictions = model.predict(X_test)
predictions_data = y_scaler.inverse_transform(predictions.reshape(-1,1))
actual_data = y_scaler.inverse_transform(y_test.reshape(-1,1))

In [34]:
print("PREDICTED DATA","\t","ACTUAL DATA")
for i in range(len(predictions_data)):
  print(predictions_data[i][0],"\t",actual_data[i][0])

PREDICTED DATA 	 ACTUAL DATA
420.19424 	 422.8999999999999
417.55722 	 427.8
420.388 	 424.15
418.8185 	 430.0
422.42697 	 425.99999999999994
420.6543 	 441.5
432.71616 	 434.75
431.05762 	 434.95
427.3493 	 448.65
439.87592 	 461.0
455.93912 	 443.7
440.442 	 437.90000000000003
427.64886 	 434.3999999999999
426.76495 	 429.8
425.69376 	 430.0
424.6457 	 427.59999999999997
421.6837 	 430.0
422.58975 	 424.85
418.8648 	 422.15000000000003
414.85986 	 415.0
408.56244 	 411.09999999999997
404.18805 	 409.49999999999994
402.90942 	 419.0
411.32578 	 414.49999999999994
410.2756 	 410.34999999999997
403.75812 	 414.4
405.75778 	 423.8500000000001
416.7038 	 427.49999999999994
422.70294 	 431.40000000000003
424.9229 	 431.25
424.5732 	 432.0
425.30762 	 431.05000000000007
425.36127 	 429.0
423.57297 	 430.25000000000006
424.15607 	 430.1
424.61172 	 435.0
428.623 	 433.0
427.74222 	 444.8500000000001
436.79953 	 468.0
460.9723 	 449.40000000000003
447.1974 	 440.15000000000003
429.5595 	 438.

In [35]:
mse_scalled = np.mean(predictions-y_test)**2
mse = np.mean(actual_data-predictions_data)**2
print()
print("MSE_SCALLED",mse_scalled)
print("MSE",mse)
print()


MSE_SCALLED 0.0005172742330879849
MSE 79.48621376094228



In [36]:
#Making the predictions value as list for the plot
predict = []
for i in predictions_data:
  predict.append(i[0])
predict

train = []
for dat in y_data_arr[:training_data_len]:
  train.append(dat[0])

test = []
for dat in y_data_arr[training_data_len:]:
  test.append(dat[0])

In [37]:
# Predict for the next x days given.
count = 7 #predict for next 7 days
next_days = []
data_list = list(y_data_arr)
while(count!=0):
  last = data_list[-time_step:]
  last_data = np.asarray(last, dtype='float32')
  last_data = last_data.reshape(-1,1)
  last_scalled = y_scaler.transform(last_data)
  x_test = []
  x_test.append(last_scalled)
  x_test = np.array(x_test)
  x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1], 1))
  prediction = model.predict(x_test)
  prediction = y_scaler.inverse_transform(prediction)
  next_days.append(prediction[0][0])
  data_list.append(prediction[0])
  count-=1


In [38]:
for i in range(len(next_days)):
  print(next_days[i])

454.85547
445.26874
436.7983
429.47437
422.60892
415.69876
408.59225


In [39]:
predict_next_days = 7
date_train = list(dates)[:training_data_len]
date_test = list(dates)[training_data_len:]
date_next = [str(i+1)+"-Day" for i in range(predict_next_days)]

In [42]:
def get_figure(title = "SBIN Stock Graph"):
  [train, test, predict, next_days], [date_train, date_test, date_test, date_next], ["Training data", "Testing data", "Tested data", "Predicted data"]
  
  y_axis = [train, test, predict, next_days]
  x_axis = [date_train, date_test, date_test, date_next]
  labels = ["Training data", "Test data Actual", "Test data Predicted", "Next 7 days Predicted data"]

  fig = go.Figure()

  if( (len(x_axis) != len(y_axis)) ):
    fig.show()

  else:
        
    for i in range(len(x_axis)):
      if(len(labels) == 0):
        fig.add_trace(go.Scatter(x=x_axis[i], y=y_axis[i],mode='lines',name=str(i+1)+"-value"))
      else:
        fig.add_trace(go.Scatter(x=x_axis[i], y=y_axis[i],mode='lines',name=labels[i]))
     
  
    fig.update_xaxes(
      rangeslider_visible=True,
      rangeselector=dict(
          buttons=list([
                        dict(count=1,
                              label='1y',
                              step='year',
                              stepmode='todate'),
                        dict(count=2,
                              label='2y',
                              step='year',
                              stepmode='todate'),
                        dict(count=3,
                              label='3y',
                              step='year',
                              stepmode='todate'),
                        dict(step='all')
          ])
      )
    )
    fig.update_layout(
      title={'text': title,
              'x':0.5,
              'xanchor': 'center',
              'yanchor': 'top'},
      xaxis_title="Date",
      yaxis_title="Data",
    )
    fig.show()

In [43]:
get_figure()